In [5]:
# from docx import Document
# from docx.oxml.ns import qn
import chardet
import re
import nltk
from nltk.tokenize import word_tokenize
# from deep_translator import GoogleTranslator
from collections import Counter
nltk.download('punkt')

[nltk_data] Error loading punkt: <urlopen error [Errno 8] nodename nor
[nltk_data]     servname provided, or not known>


False

In [6]:

class Pars_and_tokenize():

    STOP_WORDS = ['комментарии', 'примечания', 'конец ознакомительного фрагмента']
    TITLE_REGEX = '<title>(.+?)</title>'  # should be a constant
    SECTION_REGEX = '<section>(.+?)</section>'
    TAGS = ['<title>', '</title>', '<p>', '</p>', '<strong>', '</strong>',
            '\xa0', '<emphasis>', '</emphasis>', '<section>', '</section>',
            '<epigraph>', '</epigraph>', '<cite>', '</cite>', '<table>', '</table>'
            '<empty-line>', '</empty-line>', '<a>', '</a>', '<binary>', '</binary>']
    PUNCT_MARKS = ['``', '``', '"', "«", "»", "'", '.', '…', ',', '!', '?', '-', ';', ':', '(', ')',
                   '–', '—', '/', '<', '>', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
    BATCH_SIZE_TRANSLATE = 250

    def __init__(self, file):
        self.file = file
        self.num_chapters, self.content_chapters = self.chapter_extractor(self.file)

    # This function extracts number of define chapter and returns text content and number of all chapters, for russian books only
    def chapter_extractor(self, file):
        # defining the file encoding for book
        with open(self.file, 'rb') as encode_file:
            rawdata_for_encoding = encode_file.read()
            file_encoding = chardet.detect(rawdata_for_encoding)
            file_encoding = str(file_encoding['encoding'])
        chapter_count = 0
        with open(file, 'r', encoding=file_encoding) as file:
            f = file.read()
            f = f.replace('\n', '')
            sections = re.findall(self.SECTION_REGEX, f)
            print(sections[0])
            chapters = []
            # grouping by chapters
            for section in sections:
                flag_title = bool(re.findall(self.TITLE_REGEX, section))
                if flag_title and len(section) > 100:  # 100 should be a constant
                    chapter_count += 1
                    chapters.append(section)
                # For chapters which contain more than one section construct inside
                elif len(chapters) > 1:
                    chapters[-1] = chapters[-1] + section
            # remove complimentary chapters
            for word in self.STOP_WORDS:
                if word in chapters[-1].lower():
                    chapters.pop(-1)
                    chapter_count -= 1
            return chapter_count, chapters
    # return content for each chapter with all special characters
    def content(self, num_chapt):
        return self.content_chapters[num_chapt-1]

    # return content for each chapter without special characters
    def clean_content(self, num_chapt):
        text = self.content_chapters[num_chapt-1]
        for tag in self.TAGS:
            text = text.replace(tag, ' ')
        text = re.sub('<image(.+?)/>', '', text, flags=re.DOTALL)
        return text



In [7]:
book = Pars_and_tokenize('/Users/galant/Documents/Github/book_dictionary/books/498373.fb2')

 start point      <title>         <p>Александр Дюма</p>        <p>Граф де Монте-Кристо</p>      </title>      <section>point1        <title> titel1          <p>            <strong>Часть четвертая</strong>          </p>        </title>        <section>          <title>            <p>I</p>            <p>              <strong>ГОСПОДИН НУАРТЬЕ ДЕ ВИЛЬФОР</strong>            </p>          </title>          <p>Вот что произошло в доме королевского прокурора после отъезда г-жи Данглар и ее дочери, в то время как происходил переданный нами разговор.</p>          <p>Вильфор в сопровождении жены явился в комнату своего отца; что касается Валентины, то мы знаем, где она находилась.</p>          <p>Поздоровавшись со стариком и отослав Барруа, старого лакея, прослужившего у Нуартье больше четверти века, они сели.</p>          <p>Нуартье сидел в большом кресле на колесиках, куда его сажали утром и откуда поднимали вечером; перед ним было зеркало, в котором отражалась вся комната, так что, даже не ше

In [8]:
with open('/Users/galant/Documents/Github/book_dictionary/books/498373.fb2', 'rb') as encode_file:
            rawdata_for_encoding = encode_file.read()
            file_encoding = chardet.detect(rawdata_for_encoding)
            file_encoding = str(file_encoding['encoding'])
chapter_count = 0
with open('/Users/galant/Documents/Github/book_dictionary/books/498373.fb2', 'r', encoding=file_encoding) as file:
            f = file.read()
            f = f.replace('\n', '')

In [5]:
f = ' '.join(f.split())
sections = re.findall("<section>(.+?)</section>", f)

In [1]:
import xml.etree.ElementTree as ET
import os


class FB2Parser:
    def __init__(self, filename, external_annotations=True):
        self.root = ET.parse(filename).getroot()
        self.cleanup()
        self.external_annotations = external_annotations

    def cleanup(self):
        for element in self.root.iter():
                element.tag = element.tag.partition('}')[-1]

    def is_flat(self):
        return self.root.find('./body/section/section') is None
    

    def extract(self):
        self._book_title = self.root.find('./description/title-info/book-title').text
        self._main, *rest, self._annotations = self.root.findall('body')
        if self.is_flat():
            self.write(self.split(self._main))
        else:
            for part_id, part in enumerate(self._main.findall('section')):
                self.write(self.split(part, id=part_id))
        self.write_description()
        if self._annotations and self.external_annotations:
            self.write_annotations()

    def split(self, section, id=None):
        part_title = section.find('./title/p').text
        parts = {}
        for chapter_id, chapter in enumerate(section.findall('section')):
            chapter_title = chapter.find('./title/p').text
            part_path = '%02d_%s' % (id, part_title) if id is not None else ''
            chapter_path = os.path.join(part_path,
                                        '%02d_%s.fb2' % (chapter_id, chapter_title))
            path = os.path.join('.', self._book_title, chapter_path)
            parts[path] = chapter
        return parts

    def write_description(self):
        path = os.path.join('.', self._book_title, 'description.fb2')
        fb = ET.Element('FictionBook', attrib={'xmlns': "http://www.gribuser.ru/xml/fictionbook/2.0"})
        fb.append(self.root.find('description'))
        images = self.root.find('binary')
        if images:
            fb.append(images)
        book = ET.ElementTree(fb)
        book.write(path, encoding='utf-8', xml_declaration=True)

    def write_annotations(self):
        path = os.path.join('.', self._book_title, 'annotations.fb2')
        self.write({path: self._annotations})

    def write(self, data):
        for path, chapter in data.items():
            dir = os.path.dirname(path)
            fb = ET.Element('FictionBook',
                            attrib={'xmlns': "http://www.gribuser.ru/xml/fictionbook/2.0"})
            body = ET.SubElement(fb, 'body')
            body.append(chapter)
            if self._annotations and not self.external_annotations:
                body.append(self._annotations)
            book = ET.ElementTree(fb)
            os.makedirs(dir, exist_ok=True)
            book.write(path, encoding='utf-8', xml_declaration=True)



In [12]:
book1 = FB2Parser('/Users/galant/Documents/Github/book_dictionary/books/498373.fb2')

In [13]:
book_title = book1.root.find('./description/title-info/book-title').text
main, *rest= book1.root.findall('body')
branch = main


In [9]:
def innertext(tag):
  return (tag.text or '') + ''.join(innertext(e) for e in tag) + (tag.tail or '')

In [38]:
import xml.etree.ElementTree as ET
import os

class Pars_and_tokenize():
    def __init__(self, filename, external_annotations=True):
        self.root = ET.parse(filename).getroot()
        self.cleanup()
        self.external_annotations = external_annotations
        self.main = self.main_part()
        self.sections = {1:[]}
        self.chapters = {}
        self.chapters_name = {1 : []}
        self.branch = main
        self.chapter_cnt = 1
        self.cnt = 0
        self.go_into_branch(self.main)
    
    def cleanup(self):
        for element in self.root.iter():
                element.tag = element.tag.partition('}')[-1]
    
    def _clean_after_text(self,text):
        # Remove HTML tags using BeautifulSoup
        soup = BeautifulSoup(text, "html.parser")
        text = soup.get_text()
        
        # Remove special characters and extra whitespace
        text = re.sub(r'\s+', ' ', text)  # Replace multiple spaces, newlines, etc. with a single space
        text = re.sub(r'\xa0', ' ', text)  # Replace non-breaking spaces
        text = re.sub(r'\n', ' ', text)  # Remove newline characters
        text = re.sub(r'\t', ' ', text)  # Remove tab characters
        
        # Remove any remaining HTML entities
        text = re.sub(r'&[a-z]+;', '', text)
        
        return text.strip()
    # def clean_chapters(self):
    #     for key, value in self.sections.items():
    #         for part in 

    def main_part(self):
        book_title = book1.root.find('./description/title-info/book-title').text
        main, *rest= book1.root.findall('body')
        return main
    
    def innertext(self,tag):
        return (tag.text or '') + ''.join(innertext(e) for e in tag) + (tag.tail or '')
    

    def go_into_branch(self,branch):
        if not (branch.find('title') is None):
                if len(self.chapters_name[self.chapter_cnt]) == 0 or self.chapters_name[self.chapter_cnt][0] == '' or len(self.sections[self.chapter_cnt]) == 0:
                    pass
                else:
                    self.chapter_cnt += 1
                self.chapters_name[self.chapter_cnt] = self.innertext(branch.find('title'))
                self.sections[self.chapter_cnt] = []
                
        if (branch.find('section') is None):
                self.sections[self.chapter_cnt].append(innertext(branch))
                return 
        else:
            for part in branch.findall('section'):
                self.go_into_branch(part)
    

In [39]:
book2 = Pars_and_tokenize('/Users/galant/Documents/Github/book_dictionary/books/498373.fb2')

In [45]:
book2.clean_after_text(book2.sections[1][1])

'— И это они сказали тебе то, что рассердило тебя? Что же это может быть? Хочешь, я пойду спрошу их, чтобы знать, за что мне просить у тебя прощения? — Нет, нет, — ответил взгляд. — Ты меня пугаешь! Что же они могли сказать? И она задумалась. — Я догадываюсь, — сказала она, понижая голос и подходя ближе к старику. — Может быть, они говорили о моем замужестве? — Да, — ответил гневный взгляд. — Понимаю, ты сердишься за то, что я молчала. Но, видишь ли, они мне строго-настрого запретили тебе об этом говорить; они и мне ничего не говорили, и я совершенно случайно узнала эту тайну; вот почему и не была откровенна с тобой. Прости, дедушка. Взгляд, снова неподвижный и безучастный, казалось, говорил: «Меня огорчает не только твое молчание». — В чем же дело? — спросила Валентина. — Или ты думаешь, что я покину тебя, дедушка, что, выйдя замуж, я тебя забуду? — Нет, — ответил старик. — Значит, они сказали тебе, что господин д’Эпине согласен на то, чтобы мы жили вместе? — Да. — Так почему же ты се

In [14]:
sections = {1:[]}
chapters_name = {1 : []}
branch = main
chapter_cnt = 1
cnt = 0
def go_into_branch(branch):
    global cnt
    global chapter_cnt
    global chapters_name
    global sections
    if not (branch.find('title') is None):
            if len(chapters_name[chapter_cnt]) == 0 or chapters_name[chapter_cnt][0] == '' or len(sections[chapter_cnt]) == 0:
                  pass
            else:
                chapter_cnt += 1
            chapters_name[chapter_cnt] = innertext(branch.find('title'))
            sections[chapter_cnt] = []
            
    if (branch.find('section') is None):
            sections[chapter_cnt].append(innertext(branch))
            return 
    else:
        for part in branch.findall('section'):
            go_into_branch(part)

go_into_branch(branch)

In [16]:
import re
from bs4 import BeautifulSoup

def clean_text(text):
    # Remove HTML tags using BeautifulSoup
    soup = BeautifulSoup(text, "html.parser")
    text = soup.get_text()
    
    # Remove special characters and extra whitespace
    text = re.sub(r'\s+', ' ', text)  # Replace multiple spaces, newlines, etc. with a single space
    text = re.sub(r'\xa0', ' ', text)  # Replace non-breaking spaces
    text = re.sub(r'\n', ' ', text)  # Remove newline characters
    text = re.sub(r'\t', ' ', text)  # Remove tab characters
    
    # Remove any remaining HTML entities
    text = re.sub(r'&[a-z]+;', '', text)
    
    return text.strip()

# Example usage
raw_text = """
<section>
    <b>Bold Text</b> and <i>Italic Text</i>
    <div>Some content with \n newlines and \xa0 non-breaking spaces.</div>
</section>
"""

cleaned_text = clean_text(raw_text)
print(cleaned_text)

Bold Text and Italic Text Some content with newlines and non-breaking spaces.


In [27]:
sections[2]


['\n          \n            \n              II\n            \n            \n              ЗАВЕЩАНИЕ\n            \n          \n          Когда Барруа выходил из комнаты, Нуартье лукаво и многозначительно взглянул на внучку. Валентина поняла этот взгляд; понял его и Вильфор, потому что лицо его омрачилось и брови сдвинулись.\n          Он взял стул и, усевшись против паралитика, приготовился ждать.\n          Нуартье смотрел на него с полнейшим равнодушием, но уголком глаза он велел Валентине не беспокоиться и тоже оставаться в комнате.\n          Через три четверти часа Барруа вернулся вместе с нотариусом.\n          —\xa0Сударь,\xa0— сказал Вильфор, поздоровавшись с ним,\xa0— вас вызвал присутствующий здесь господин Нуартье де Вильфор; общий паралич лишил его движения и голоса, и только мы одни, и то с большим трудом, умудряемся понимать кое-какие обрывки его мыслей.\n          Нуартье обратил на Валентину свой взгляд, такой серьезный и властный, что она немедленно вступилась:\n      

In [24]:
clean_text(sections[1][0]).index(r'\')

ValueError: substring not found